In [ ]:
import random as rd
import time
import pandas as pd

###################### classes ##################################################################

class Deck():
    
    def __init__(self):
        global global_card_dict
        global_card_dict = {'cardID':list(range(52)),'cardRank':['2','3','4','5','6','7','8','9','10','J','Q','K','A','2','3','4','5','6','7','8','9','10','J','Q','K','A','2','3','4','5','6','7','8','9','10','J','Q','K','A','2','3','4','5','6','7','8','9','10','J','Q','K','A'],'cardSuit':['Spades','Spades','Spades','Spades','Spades','Spades','Spades','Spades','Spades','Spades','Spades','Spades','Spades','Clubs','Clubs','Clubs','Clubs','Clubs','Clubs','Clubs','Clubs','Clubs','Clubs','Clubs','Clubs','Clubs','Diamonds','Diamonds','Diamonds','Diamonds','Diamonds','Diamonds','Diamonds','Diamonds','Diamonds','Diamonds','Diamonds','Diamonds','Diamonds','Hearts','Hearts','Hearts','Hearts','Hearts','Hearts','Hearts','Hearts','Hearts','Hearts','Hearts','Hearts','Hearts']}
    
    def shuffle(self):
        self.deck = rd.sample(list(range(52)),52)
        self.outOfDeck = []
    
    def dealCards(self,numCards):
        cardsDealt = self.deck[0:numCards]
        self.outOfDeck += self.deck[0:numCards]
        del self.deck[0:numCards]
        return cardsDealt
    
class Hand():

    def __init__(self):
        self.cardsInHand = []

    def draw(self,cardsDrawn):
        self.cardsInHand += cardsDrawn    
        

    def score(self,dealer=0,mute=0):
  ### Scores someone's hand. 
  ## dealer = 1 hides scoring for dealer and "hides" the lose function trigger
        value = 0
        numAces = 0
        for a in self.cardsInHand:
            if global_card_dict['cardRank'][a] in ['J','Q','K']:
                value = value + 10
            elif global_card_dict['cardRank'][a] == 'A': 
                value = value + 1
                numAces += 1
            else: 
                value = value + int(global_card_dict['cardRank'][a])
        
        if numAces > 0 and value+10 <= 21 :
          ##value+10, if the the player score is <=21 if one of the Ace(s) is considered an 11, consider one of the Aces an 11    
            value = value+10

        if dealer == 0 and mute != 1:
            print("I have a score of " + str(value)+".")

        if dealer == 0 and value > 21:
            print('I went over 21....')
            lose()

        return value


class Cards():
    def readHand(hand):
        ## Prints the names of all of the cards in the hand. 
        for cards in hand:
            print('   The ' + global_card_dict['cardRank'][cards] + ' of ' + global_card_dict['cardSuit'][cards])

class Record():
    def __init__(self):
        self.log = []
        try:
            self.df = pd.read_csv('heh theres nothing')

        except FileNotFoundError:
            initialize = { 'game_id': [], 
                           'turn_number': [],
                           'player_hand': [],
                           'dealer_hand': []
                         }
    
            self.df = pd.DataFrame(initialize)
            print(self.df)
            
    def appendRow(self,gameId, turn_number, playerHand,dealerHand):
        self.df.loc[len(self.df)] = [gameId,turn_number,'','']
        self.df.iat[len(self.df)-1,2] = playerHand
        self.df.iat[len(self.df)-1,3] = dealerHand
        print(self.df)

################################# Post Game Functions #########################################################
def win():
    print('I win!')
    playAgain()

def lose():
    print('I lose. :(')
    playAgain()
    
    
def playAgain(recursive=0):
### Use this function after a game is done.
### To do - Add more features

    if recursive==0:
        print('''Play again? Please enter 'Y' for yes or 'N' for no.''')
    
    decide = input()

    if decide not in ['Y','N','y','n']:
        print('''Invalid input. Please enter 'Y' for yes or 'N' for no.''')
        playAgain(1)    
    elif decide in ['Y','y']:
        print('Shuffling cards.....')
        time.sleep(1) 
        gameStart()
    elif decide in ['N','n']:
        print('Thanks for playing! :)')
        return
    
################ Gameplay Engine ################################################################################

def gameStart():
    """Initializes a game of blackjack. Deals the cards to the player and dealer, then passes the player hand, dealer hand, and deck to hitOrStay()
    
    """
    
    d = Deck()
    logger = Record()
    
    
    d.shuffle()
    player1 = Hand()
    dealer = Hand()
    player1.draw(d.dealCards(2))
    print('I receive 2 cards. My hand contains:')
    Cards.readHand(player1.cardsInHand)
    player1.score()
    print('The dealer receives 2 cards.')
    dealer.draw(d.dealCards(2))
    print('''The dealer's first card is the ''' + global_card_dict['cardRank'][dealer.cardsInHand[0]]+ ' of ' + global_card_dict['cardSuit'][dealer.cardsInHand[0]] + ', their second card is face down.')
    
    logger.appendRow(1,2,player1.cardsInHand,dealer.cardsInHand)
    hitOrStay(player1,dealer,d)
    
    
    
def hitOrStay(playerHand,dealerHand, deck, mute=0):
    """Returns H or S depending on if the player wants to stand or hit.
    """
    
    if mute == 0:
        print('''What do you want to do? Input 'S' to stand and 'H' to hit. ''')
    
    decide = input()

    
    if decide not in ['S','H','s','h']:
        print('''Invalid input. Please enter either 'S' to stand and 'H' to hit. ''')
        hitOrStay(playerHand,dealerHand,deck,mute=1)    
    elif decide in ['S','s']:
        stay(playerHand,dealerHand, deck)
    elif decide in ['H','h']:
        hit(playerHand,dealerHand, deck)

########################################################################################################
def hit(playerHand,dealerHand,deck):
    """ Adds another card to the player's hand, and reprompts them to hit or stay if they didn't bust.
    """
    
    print('I decide to take a hit, and draw:')
    playerHand.draw(deck.dealCards(1))
    Cards.readHand([playerHand.cardsInHand[-1]])
    print('My hand now contains:')
    Cards.readHand(playerHand.cardsInHand)
    playerHand.score()
    hitOrStay(playerHand,dealerHand,deck)
    

def stay(playerHand,dealerHand,deck):
    """ Prompts the dealer's turn once the player is done drawing.
    
    """
    print('The dealer reveals their face down card as:')
    Cards.readHand([dealerHand.cardsInHand[1]])
    print('''The dealer's hand is:''')
    Cards.readHand(dealerHand.cardsInHand)
    dealerScore = dealerHand.score(dealer=1, mute=1)
    print('''The dealer's score is ''' + str(dealerScore) + '.')
    dealerDraw(playerHand,dealerHand,deck) 

##############################################################################################################

def dealerDraw(playerHand,dealerHand,deck):
    """After the player stays, the dealer will draw cards as dictated by the rules of Blackjack. 
       If their score is below 17, they must draw, and if they are between 17 and 21 inclusive, they must stand. 
       Scoring above 21 is a bust.
    
       Args: playerHand (class '__main__.Hand')
             dealerHand (class '__main__.Hand')
             deck       (class '__main__.Deck')
    
    """
    dealerScore = dealerHand.score(dealer=1, mute=1)
    
    if dealerScore <= 16:
        print('Because their score is below 17, they must take another card.')
        dealerHand.draw(deck.dealCards(1))
        print('The dealer draws:')
        Cards.readHand([dealerHand.cardsInHand[-1]])
        dealerScore = dealerHand.score(dealer=1, mute=1)
        print('''The dealer now has a score of: '''+ str(dealerScore) + ''' and the dealer's hand is now:''')
        Cards.readHand(dealerHand.cardsInHand)
        
        dealerDraw(playerHand,dealerHand,deck)
        
    elif dealerScore > 16 and dealerScore <= 21:
        print('The dealer stands with a score of: ' + str(dealerScore))
        compare(playerHand,dealerHand)
        
    elif dealerScore > 21:
        print('The dealer went over 21!')
        win()
        
def compare(playerHand,dealerHand):
    """Compares the final player hand to the final dealer hand if both parties did not bust.
    
    """
    dealerScore = dealerHand.score(dealer=1, mute=1)
    playerScore = playerHand.score(mute=1)
    
    if playerScore > dealerScore:
        print('My score (' + str(playerScore) + ''') is higher that the dealer's score ('''+ str(dealerScore) + ').')
        win()
    elif playerScore < dealerScore:
        print('My score (' + str(playerScore) + ''') is lower than the dealer's score (''' + str(dealerScore) + ').')
        lose()
    elif playerScore == dealerScore:
        print('We have the same score (' + str(playerScore) + ') , so we have tied.')
        playAgain()
###############################################################################################################

gameStart()


Empty DataFrame
Columns: [game_id, turn_number, player_hand, dealer_hand]
Index: []
I receive 2 cards. My hand contains:
   The 2 of Clubs
   The 9 of Diamonds
I have a score of 11.
The dealer receives 2 cards.
The dealer's first card is the 3 of Diamonds, their second card is face down.
   game_id  turn_number player_hand dealer_hand
0      1.0          2.0    [13, 33]    [27, 16]
What do you want to do? Input 'S' to stand and 'H' to hit. 


In [3]:
import pandas as pd

try:
    df = pd.read_csv('heh theres nothing')

except FileNotFoundError:
    print('gottem')
    initialize = { 'game_id': [], 
                   'turn_number': [],
                   'player_hand': [],
                   'dealer_hand': []
                 }
    
    df = pd.DataFrame(initialize)
    turn = [1,2,[3,4],4]
    df.loc[len(df)] = turn
    print(df)
        


gottem
   game_id  turn_number player_hand  dealer_hand
0      1.0          2.0      [3, 4]          4.0


c:\python38\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [3]:
import pandas as pd
import random as rd

class Record():
    
    def __init__(self):
        try:
            self.df = pd.read_csv('heh theres nothing')

        except FileNotFoundError:
            print('gottem')
            initialize = { 'turn_number': [],
                           'score':[]
                         }

            self.df = pd.DataFrame(initialize)
            
    def appendRow(self, roll_number, score):
        self.df.loc[len(self.df)] = [roll_number, score]
        
        

def roll(num):
    score = 0
    for roll in range(num):
        score += rd.randrange(6)+1
    return score    

def game():
    logger = Record()
    play = True
    roll_num = 1
    while play == True:
        playerScore = roll(3)
        roll_num += 1
        logger.appendRow(roll_num,playerScore)
        print(logger.df)
        
        if roll_num == 10:
            play = False
        
game()


    

gottem
   turn_number  score
0          2.0   15.0
   turn_number  score
0          2.0   15.0
1          3.0   11.0
   turn_number  score
0          2.0   15.0
1          3.0   11.0
2          4.0   13.0
   turn_number  score
0          2.0   15.0
1          3.0   11.0
2          4.0   13.0
3          5.0    8.0
   turn_number  score
0          2.0   15.0
1          3.0   11.0
2          4.0   13.0
3          5.0    8.0
4          6.0   10.0
   turn_number  score
0          2.0   15.0
1          3.0   11.0
2          4.0   13.0
3          5.0    8.0
4          6.0   10.0
5          7.0    6.0
   turn_number  score
0          2.0   15.0
1          3.0   11.0
2          4.0   13.0
3          5.0    8.0
4          6.0   10.0
5          7.0    6.0
6          8.0   14.0
   turn_number  score
0          2.0   15.0
1          3.0   11.0
2          4.0   13.0
3          5.0    8.0
4          6.0   10.0
5          7.0    6.0
6          8.0   14.0
7          9.0    8.0
   turn_number  score
0  

In [4]:
import random as rd

print(rd.randrange(6))
print(rd.randrange(6))
print(rd.randrange(6))
print(rd.randrange(6))
print(rd.randrange(6))
print(rd.randrange(6))
print(rd.randrange(6))
print(rd.randrange(6))
print(rd.randrange(6))
print(rd.randrange(6))

2
1
3
2
5
0
2
4
4
2
